In [1]:
!pip install openai

You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


In [11]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [15]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        # {"role": "user", "content": "Hi"},
        {"role": "user", "content": "What is the weather like in Boston?"},
    ],
    functions=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
                },
                "required": ["location"]
            }
        }
    ],
    function_call="auto"
)
res

<OpenAIObject chat.completion id=chatcmpl-8DFMDZzxNu9kevwZ7ONi7BrK7oO3j at 0x2c29fc50b30> JSON: {
  "id": "chatcmpl-8DFMDZzxNu9kevwZ7ONi7BrK7oO3j",
  "object": "chat.completion",
  "created": 1698167989,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  }
}